In [22]:

#Carregar as bibliotecsa


# Using the community model
# BERT Base
#tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
#model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

# BERT Large
#tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
#model = AutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased')
#max_seq_length = 65000

# or, using BertModel and BertTokenizer directly
#from transformers import BertModel, BertTokenizer
#tokenizer = BertTokenizer.from_pretrained('path/to/vocab.txt', do_lower_case=False)
#model = BertModel.from_pretrained('path/to/bert_dir')  # Or other BERT model class

#from transformers import BertTokenizer, BertForQuestionAnswering
#import torch
#from transformers import AutoModel, AutoTokenizer
#import torch
#import torch.nn.functional as F
#from transformers import AutoTokenizer  # Or BertTokenizer
#from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
#from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
#import PyPDF2
#import re
#import spacy
#import numpy as np

from transformers import LongformerModel, LongformerTokenizer



from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoModelForPreTraining, BertForQuestionAnswering, AutoModel, AutoTokenizer, BertTokenizer, BertForQuestionAnswering, BertForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn.functional as F
import PyPDF2
import re
import numpy as np

import spacy

# Carregar o tokenizer e o modelo pré-treinado
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerModel.from_pretrained('allenai/longformer-base-4096')
#Modelo para perguntas
model_forquestion = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
#Modelo para resumo
modelsum = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizersum = T5Tokenizer.from_pretrained('t5-base')

def convert_pdf (doc):
    with open(doc, "rb") as f:
        # Use a biblioteca PyPDF2 para ler o arquivo
        #reader = PyPDF2.PdfFileReader(f)
        reader = PyPDF2.PdfReader(f)
        # Obtenha o número de páginas no arquivo
        #num_pages = reader.getNumPages()
        num_pages = len(reader.pages)
        # Crie uma string vazia para armazenar o texto extraído
        text = ''
        # Itere pelas páginas e extraia o texto
        for page in range(num_pages):
            # Obtenha o objeto da página
            #page_obj = reader.getPage(page)
            page_obj = reader.pages[page]
            # Extraia o texto da página
            #page_text = page_obj.extractText() 
            page_text = page_obj.extract_text()
            # Adicione o texto extraído à string de texto
            text += page_text
    return text
            

def remover_stopwords(tokens):
    # Carrega o modelo do SpaCy para o idioma português
    nlp = spacy.load('pt_core_news_lg')

    # Cria uma lista para armazenar os tokens sem stopwords
    tokens_sem_stopwords = []

    # Percorre cada token na lista
    for token in tokens:
        # Verifica se o token não é uma stopword
        if not nlp.vocab[token].is_stop:
            tokens_sem_stopwords.append(token)
    return tokens_sem_stopwords


def normalize_text(text):
   # text=text
    # Converte o texto para minúsculas
    normalized_text = text.lower()
    # Remove caracteres especiais e acentuação
    normalized_text = re.sub(r'[^\w\s]', '', normalized_text)
    # Retorna o texto limpo
    return normalized_text

def tokengen(text):
    #text=text
    max_length=4096
    # Dividir o texto em partes menores
    input_parts = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    tokenized_parts = []

    for part in input_parts:
        # Tokenizar cada parte do texto
        tokens = tokenizer.encode(part, add_special_tokens=True, truncation=True, max_length=max_length, padding='longest', return_tensors='pt')
        tokenized_parts.append(tokens)

    # Concatenar as partes tokenizadas em um único tensor
    concatenated_tokens = torch.cat(tokenized_parts, dim=1)

    # Obter a lista de tokens
    token_list = tokenizer.convert_ids_to_tokens(concatenated_tokens.flatten().tolist())
    print("Token =", len(token_list))
    print("Lista de tokens =", token_list)
    return token_list, len(token_list)


def vector_one(text):
    max_length = 4096
    average_vector = []
    vectors = []
    input_parts = []
    input_parts = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    # Lista para armazenar os vetores médios
    average_vector = []
    # Processa cada parte do texto
    for part in input_parts:
    # Tokeniza a parte do texto
        tokens = tokenizer.encode(part, add_special_tokens=True, truncation=True, max_length=4096, padding='longest', return_tensors='pt')
        #input_ids = torch.tensor(tokens)
        input_ids = tokens.clone().detach()
        # Gera os vetores de embeddings
        with torch.no_grad():
            outputs = model(tokens)
            last_hidden_states = outputs[0]
        # Calcula o vetor médio
        average_vector = torch.mean(last_hidden_states, dim=1)
        # Adiciona o vetor médio à lista
        vectors.append(average_vector)
    # Concatena os vetores gerados
    text_vector = torch.cat(vectors, dim=1)
    return text_vector
    
def similarity_vector(a, b):
    size_a = a.size(1)
    size_b = b.size(1)
    if size_a == size_b:
        similarity_score = cosine_similarity(a, b)
        return similarity_score
    if size_a > size_b:
        diff = size_a - size_b
        padded = F.pad(b, pad=(0, diff), mode='constant', value=0)
        similarity_score = cosine_similarity(a, padded)
        return similarity_score
    if size_a < size_b:
        diff = size_b - size_a
        padded = F.pad(a, pad=(0, diff), mode='constant', value=0)
        similarity_score = cosine_similarity(b, padded)
        return similarity_score
    

def question(text, question):
    question = question
    max_answer_length = 512
    max_length = 512
    input_parts = []
    input_parts = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    # Lista para armazenar as respostas
    answers = []
    # Processa cada parte do texto
    for part in input_parts:
        # Tokeniza a parte do texto
        # Tokeniza a pergunta e o texto menor
        encoding = tokenizer.encode_plus(question, part, return_tensors='pt', max_length=512, truncation=True)
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        # Obtém as respostas do modelo pré-treinado
        with torch.no_grad():
            outputs = model_forquestion(input_ids=input_ids, attention_mask=attention_mask)
            start_scores = outputs.start_logits
            end_scores = outputs.end_logits
        # Obtém a resposta com maior probabilidade
        answer_start = torch.argmax(start_scores)
        answer_end = torch.argmax(end_scores)
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start.item():answer_end.item()+1]))
        # Adiciona a resposta à lista
        answers.append(answer)
    # Combina as respostas
    combined_answer = ' '.join(answers)
    # Limita o tamanho da resposta final
    combined_answer = combined_answer[:max_answer_length]
        # Formatação da resposta final
    formatted_answer = "RESPOSTA:\n\n"
    formatted_answer += combined_answer.replace(".", ".\n\n")
    # Apresenta a resposta final
    return formatted_answer
    # Apresenta a resposta final
    return combined_answer


def summarization(text):
    num_sentences=5
    #max_length=512
    # Tokenizar o texto em partes
    input_parts = text.split('. ')
    summaries = []
    #input_parts = []
    #input_parts = [text[i:i+max_length] for i in range(0, len(text), max_length)]    
    for part in input_parts:
        lenght=len(part)
        m_length = int(lenght*0.4)
        # Tokenizar a parte do texto
        inputs = tokenizersum.encode("summarize: " + part, return_tensors="pt")
        # Gerar o resumo da parte do texto usando o modelo T5
        outputs = modelsum.generate(inputs, max_length=m_length, num_beams=4, early_stopping=True)
        summary = tokenizersum.decode(outputs[0], skip_special_tokens=True)

        # Adicionar o resumo à lista de sumários
        summaries.append(summary)

    # Concatenar os sumários em um único texto
    concatenated_summary = ' '.join(summaries)

    # Extrair as N sentenças mais importantes
    sentences = concatenated_summary.split('. ')
    top_sentences = sorted(sentences, key=lambda x: len(x), reverse=True)[:num_sentences]

    return '. '.join(top_sentences)

def prep_proc(text1,text2):
    nm1=normalize_text(text1)
    nm2=normalize_text(text2)
    print("Texto normalizado 1: " + nm1[:200])
    print("Texto normalizado 2: " + nm2[:200])
    print("Tokens texto 1:")
    tk1=tokengen(text1)
    print("Tokens texto 2:")
    tk2=tokengen(text2)
    vt1=vector_one(nm1)
    vt2=vector_one(nm2)
    print("Vetor do artigo 1:")
    print(vt1.shape)
    print(vt1)
    print("Vetor do artigo 2:")
    print(vt2.shape)
    print(vt2)
    print("calculo de similaridades entre texto 1 e texto2 (cosseno)")
    sim=similarity_vector(vt1,vt2)
    print(sim)
    return vt1,vt2,sim

  

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\thgcn\anaconda3\lib\site-packages\transformers\models\t5\tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking

In [23]:
#carrega artigo
caminho_arquivo='C:/Users/thgcn/OneDrive/Academico/PO-245/Projeto/artigo.txt'
with open(caminho_arquivo, "r") as arquivo:
    # Ler todo o conteúdo do arquivo
    text1 = arquivo.read()
    text2 = arquivo.read()
text2=text1
prep_proc(text1,text2)


Texto normalizado 1: o projeto brazilian safety desenvolveu diversas ações ao redor do mundo para que as empresas apoiadas possam promover seus produtos
no ano de 2022 o projeto brazilian safety fruto da aliança da animas
Texto normalizado 1: o projeto brazilian safety desenvolveu diversas ações ao redor do mundo para que as empresas apoiadas possam promover seus produtos
no ano de 2022 o projeto brazilian safety fruto da aliança da animas
Token texto 1:
Token = 8706
Lista de tokens = ['<s>', 'O', 'ĠPRO', 'J', 'ET', 'O', 'ĠBRA', 'Z', 'IL', 'IAN', 'ĠSAF', 'ET', 'Y', 'ĠDES', 'EN', 'VOL', 'VE', 'U', 'ĠD', 'IVERS', 'AS', 'ĠA', 'Ã', 'ĩ', 'Ã', 'ķ', 'ES', 'ĠA', 'O', 'ĠRED', 'OR', 'ĠDO', 'ĠM', 'UN', 'DO', 'ĠP', 'ARA', 'ĠQ', 'UE', 'ĠAS', 'ĠEM', 'PRES', 'AS', 'ĠAP', 'O', 'I', 'AD', 'AS', 'ĠP', 'OSS', 'AM', 'ĠPR', 'OM', 'OVER', 'ĠSE', 'US', 'ĠPR', 'OD', 'UT', 'OS', '.', 'Ċ', 'NO', 'ĠAN', 'O', 'ĠDE', 'Ġ2022', ',', 'ĠO', 'ĠPRO', 'J', 'ET', 'O', 'ĠBRA', 'Z', 'IL', 'IAN', 'ĠSAF', 'ET', 'Y', 'ĠFR', 'U

(tensor([[ 0.0034,  0.0626,  0.0807,  ..., -0.0939,  0.0476,  0.1834]]),
 tensor([[ 0.0034,  0.0626,  0.0807,  ..., -0.0939,  0.0476,  0.1834]]),
 array([[0.9999993]], dtype=float32))